In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sys, os, caffe, cv2, copy, cPickle, gzip
from prettytable import PrettyTable
import warnings

warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (5, 5)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

caffe.set_mode_gpu()
caffe.set_device(3)

In [2]:
def read_mean_file(path):
    
    blob_mean_file = caffe.proto.caffe_pb2.BlobProto()
    mean_file = open(path, 'rb' ).read()
    blob_mean_file.ParseFromString(mean_file)
    mean_npy = np.array(caffe.io.blobproto_to_array(blob_mean_file)) # ASSUMED IN BGR
    
    return mean_npy

In [3]:
def read_image(path):

    img = cv2.imread(path, 1)

    return img  


def print_image(img_rescaled):

    img_plt = copy.copy(img_rescaled)
    channel_0 = copy.copy(img_plt[:, :, 0])
    img_plt[:, :, 0] = img_plt[:, :, 2]
    img_plt[:, :, 2] = channel_0 

    plt.imshow(img_plt)
    plt.show()


def rescale_image(img, scale = [256, 256]):

    img_rescaled = cv2.resize(img, (scale[0], scale[1]), interpolation = cv2.INTER_LINEAR) # IMAGE IS BGR

    return img_rescaled


def convert_img_shape_for_caffe(img_rescaled):

    caffe_img = np.expand_dims(img_rescaled.swapaxes(0, 2).swapaxes(1, 2), axis = 0)

    return caffe_img


def subtract_mean_from_image(caffe_img, mean_npy):

    img_mean_subtracted = caffe_img.astype('float32') - mean_npy

    return img_mean_subtracted

In [4]:
def classify_images(net, cropped_images):
    
    net.blobs['data'].data[...] = cropped_images
    out = net.forward()
    
    #print("Predicted class is #{}.".format(out['prob'][0].argmax()))
    
    return out

In [5]:
def get_predictions(out, labels, top_n, out_key):

    top_n_predictions = []
    
    predictions = out[out_key]
    
    for i in xrange(len(predictions)):
    
        predicted_indexes =  np.argsort(predictions[i])[-top_n:][::-1]
        predicted_probabilities = predictions[i][predicted_indexes].reshape(-1, 1)
        predicted_labels = labels[predicted_indexes].reshape(-1, 1)

        top_n_predictions.append(np.concatenate((predicted_labels, predicted_probabilities), axis = 1))
    
    top_n_predictions = np.array(top_n_predictions)
    
    return top_n_predictions

In [6]:
def print_predictions(top_n_predictions, title):
    
    prediction_table = PrettyTable(top_n_predictions.dtype.names)

    for row in top_n_predictions:
        prediction_table.add_row(row)

    prediction_table.field_names = ['Predicted Class', 'Predicted Probability']

    print '\n--------------{}--------------'.format(title)
    print prediction_table

In [7]:
test_image_names_from_file = np.loadtxt('/home/nvidia/imagenet2015_onur/test/test_image_names.txt', dtype = 'str')

#crop_positions_from_file = cPickle.load(gzip.open('./Test_Localization_SLIC_Outputs_oversegmented_sigma_5.pkl.gz'))
crop_positions_from_file = cPickle.load(gzip.open('./Test_Localization_SLIC_Outputs.pkl.gz'))
crop_positions_from_file = np.array(crop_positions_from_file)

predictions_from_file = cPickle.load(gzip.open('./Test_Classification_25_crops_with_Scan_20151111-045327-bcce_resume_training_after_epoch_2_with_caffe-snapshot_iter_45921.caffemodel.pkl.gz'))

print 'Shape of predictions    :', predictions_from_file.shape
print 'Shape of crop positions :', crop_positions_from_file.shape

Shape of predictions    : (3, 100000, 5, 2)
Shape of crop positions : (100000, 2)


In [ ]:
top_n = 5

labels3 = np.loadtxt('/home/nvidia/imagenet2015_onur/level3_labels/labels.txt', dtype = 'str')
labels2 = np.loadtxt('/home/nvidia/imagenet2015_onur/level2_labels/level_2_labels.txt', dtype = 'str')
labels1 = np.loadtxt('/home/nvidia/imagenet2015_onur/level1_labels/level_1_labels.txt', dtype = 'str')

node_names = np.loadtxt('../imagenet2015_onur/all_labels/constructed_labels.csv', dtype = 'str', delimiter = ',')
node_names = node_names[:, [0, 2, 3]]
node_names[:,[0, 1, 2]] = node_names[:,[1, 2, 0]]

mean_npy = read_mean_file('/home/nvidia/imagenet_lmdb/mean.binaryproto')

net = caffe.Net('/home/nvidia/digits-2.2/digits/jobs/20151105-150238-61f1/deploy_corrected.prototxt',
                '/home/nvidia/digits-2.2/digits/jobs/20151111-045327-bcce_resume_training_after_epoch_2_with_caffe/snapshot_iter_45921.caffemodel',
                caffe.TEST)

In [ ]:
bboxes_for_all_images = []
node_names_level3_for_all_images = []

for __index in xrange(len(test_image_names_from_file)):
    
    if __index % 5000 == 0:
        print 'Done :', __index
    
    img = read_image('../data/ILSVRC2015/Data/CLS-LOC/test/' + test_image_names_from_file[__index])
    img_rescaled = rescale_image(img, scale = [256, 256])
    caffe_img = convert_img_shape_for_caffe(img_rescaled)
    img_mean_subtracted = subtract_mean_from_image(caffe_img, mean_npy)

    crop_positions_for_an_image = np.array(crop_positions_from_file[__index][1])
    predictions_for_an_image = predictions_from_file[:, __index]

    #############-----------------new---------------------------------
    node_names_for_an_image = []

    for pfai in predictions_for_an_image[-1][:, 0]:
        node_names_for_an_image.append(node_names[np.where(node_names[:, -1] == pfai)[0]][0])

    node_names_for_an_image = np.array(node_names_for_an_image)
    #############--------------------------------

    
    
    
    
    cropped_mean_subtracted_images_for_an_image = []
    cropped_rescaled_images_for_an_image = []

    for cpfai in crop_positions_for_an_image:
        _cropped = img_mean_subtracted[:, :, cpfai[0]:cpfai[1], cpfai[2]:cpfai[3]]
        _cropped = np.squeeze(_cropped, axis = 0).swapaxes(0, 2).swapaxes(0, 1)

        _cropped_rescaled = rescale_image(_cropped, scale = [224, 224])
        _cropped_rescaled = _cropped_rescaled.swapaxes(0, 2).swapaxes(1, 2)

        cropped_mean_subtracted_images_for_an_image.append(_cropped_rescaled)

        cropped_rescaled_images_for_an_image.append(rescale_image(img_rescaled[cpfai[0]:cpfai[1], cpfai[2]:cpfai[3]], 
                                                                  scale = [224, 224]))

    cropped_mean_subtracted_images_for_an_image = np.array(cropped_mean_subtracted_images_for_an_image)
    cropped_rescaled_images_for_an_image = np.array(cropped_rescaled_images_for_an_image)
    
    
    
    
    net.blobs['data'].reshape(cropped_mean_subtracted_images_for_an_image.shape[0],
                              cropped_mean_subtracted_images_for_an_image.shape[1],
                              cropped_mean_subtracted_images_for_an_image.shape[2],
                              cropped_mean_subtracted_images_for_an_image.shape[3])

    caffe_output = classify_images(net, cropped_mean_subtracted_images_for_an_image)

    top_n_predictions_level1 = get_predictions(caffe_output, labels1, top_n, 'prob1')
    top_n_predictions_level2 = get_predictions(caffe_output, labels2, top_n, 'prob2')
    top_n_predictions_level3 = get_predictions(caffe_output, labels3, top_n, 'prob3')
    
    
    
    
    node_names_for_level1 = np.array(list(set(node_names_for_an_image[:, 0])))
    node_names_for_level2 = np.array(list(set(node_names_for_an_image[:, 1])))
    node_names_for_level3 = np.array(list(set(node_names_for_an_image[:, 2])))
    
    
    
    
    
    crop_indexes_for_level1 = []

    for i in xrange(len(node_names_for_level1)):

        for k in xrange(5):

            out_index = list(np.where(top_n_predictions_level1[:, k, 0] == node_names_for_level1[i])[0])

            if out_index:
                break

        if not out_index:
            out_index = None

        crop_indexes_for_level1.append(out_index)

    crop_indexes_for_level1 = np.array(crop_indexes_for_level1)

    
    
    
    parent_nodes_for_level2 = []

    for i in xrange(len(node_names_for_level2)):
        parent_nodes_for_level2.append(node_names_for_an_image[np.where(node_names_for_an_image[:, 1] == node_names_for_level2[i])[0]][:, 0][0])

    parent_nodes_for_level2 = np.array(parent_nodes_for_level2)


    
    
    
    crop_indexes_for_level2 = []

    for i in xrange(len(node_names_for_level2)):

        avaliable_crops = crop_indexes_for_level1[np.where(node_names_for_level1 == parent_nodes_for_level2[i])[0]][0]

        if avaliable_crops != None:

            avaliable_crops_numpy = np.array(avaliable_crops)

            for k in xrange(5):

                out_index = list(np.where(top_n_predictions_level2[avaliable_crops_numpy, k, 0] == node_names_for_level2[i])[0])

                if out_index:
                    out_index = list(avaliable_crops_numpy[out_index])
                    break

            if not out_index:
                out_index = None

        if avaliable_crops == None or out_index == None:

            for k in xrange(5):

                out_index = list(np.where(top_n_predictions_level2[:, k, 0] == node_names_for_level2[i])[0])

                if out_index:
                    break

            if not out_index:
                out_index = None

        crop_indexes_for_level2.append(out_index)

    crop_indexes_for_level2 = np.array(crop_indexes_for_level2)


    
    
    
    parent_nodes_for_level3 = []

    for i in xrange(len(node_names_for_level3)):
        parent_nodes_for_level3.append(node_names_for_an_image[np.where(node_names_for_an_image[:, 2] == node_names_for_level3[i])[0]][:, 1][0])

    parent_nodes_for_level3 = np.array(parent_nodes_for_level3)


    
    
    crop_indexes_for_level3 = []

    for i in xrange(len(node_names_for_level3)):

        avaliable_crops = crop_indexes_for_level2[np.where(node_names_for_level2 == parent_nodes_for_level3[i])[0]][0]

        if avaliable_crops != None:

            avaliable_crops_numpy = np.array(avaliable_crops)

            for k in xrange(5):

                out_index = list(np.where(top_n_predictions_level3[avaliable_crops_numpy, k, 0] == node_names_for_level3[i])[0])

                if out_index:
                    out_index = list(avaliable_crops_numpy[out_index])
                    break

            if not out_index:
                out_index = None

        if avaliable_crops == None or out_index == None:

            nnfl1 = parent_nodes_for_level2[np.where(node_names_for_level2 == parent_nodes_for_level3[i])[0]]

            avaliable_crops = crop_indexes_for_level1[np.where(node_names_for_level1 == nnfl1)[0]][0]

            if avaliable_crops != None:

                avaliable_crops_numpy = np.array(avaliable_crops)

                for k in xrange(5):

                    out_index = list(np.where(top_n_predictions_level3[avaliable_crops_numpy, k, 0] == node_names_for_level3[i])[0])

                    if out_index:
                        out_index = list(avaliable_crops_numpy[out_index])
                        break

                if not out_index:
                    out_index = None

            if avaliable_crops == None or out_index == None:

                for k in xrange(5):

                    out_index = list(np.where(top_n_predictions_level3[:, k, 0] == node_names_for_level3[i])[0])

                    if out_index:
                        break

                if not out_index:
                    out_index = None

        crop_indexes_for_level3.append(out_index)

    crop_indexes_for_level3 = np.array(crop_indexes_for_level3)


    
    
    
    
    predicted_bounding_boxes_for_an_image = []

    for i in xrange(len(crop_indexes_for_level3)):

        if crop_indexes_for_level3[i] is not None:
            bbox = list(np.round(np.mean(crop_positions_for_an_image[crop_indexes_for_level3[i]], axis = 0)).astype('int'))
           
        else:
            bbox = None

        predicted_bounding_boxes_for_an_image.append(bbox)

    #predicted_bounding_boxes_for_an_image = np.array(predicted_bounding_boxes_for_an_image)


    bboxes_for_all_images.append(predicted_bounding_boxes_for_an_image)
    node_names_level3_for_all_images.append(node_names_for_level3)
    
    
#bboxes_for_all_images = np.array(bboxes_for_all_images)
node_names_level3_for_all_images = np.array(node_names_level3_for_all_images)


In [22]:
#node_names_level3_for_all_images
#bboxes_for_all_images

In [18]:
print bboxes_for_all_images[1]
print node_names_level3_for_all_images[1]

[None, [29, 250, 2, 221], [139, 244, 3, 144], [44, 216, 67, 237], [166, 254, 132, 254]]
['conch' 'eel' 'slug' 'banana' 'fig']


In [21]:
f = gzip.open('./Test_Localization_Segment_Bounding_Boxes_with_SLIC_New_Procedure_with_LOC-Test_Localization_SLIC_Outputs-and-CLS-Test_Classification_25_crops_with_Scan_20151111-045327-bcce_resume_training_after_epoch_2_with_caffe-snapshot_iter_45921.caffemodel.pkl.gz', 'w')

In [22]:
cPickle.dump((node_names_level3_for_all_images, bboxes_for_all_images), f)
f.close()

In [ ]:
__index = 82552 #82152 #62132 #64278

img = read_image('../data/ILSVRC2015/Data/CLS-LOC/test/' + test_image_names_from_file[__index])
img_rescaled = rescale_image(img, scale = [256, 256])
caffe_img = convert_img_shape_for_caffe(img_rescaled)
img_mean_subtracted = subtract_mean_from_image(caffe_img, mean_npy)

#print_image(img_rescaled)

crop_positions_for_an_image = np.array(crop_positions_from_file[__index][1])
predictions_for_an_image = predictions_from_file[:, __index]

#############-----------------new---------------------------------
node_names_for_an_image = []

for pfai in predictions_for_an_image[-1][:, 0]:
    node_names_for_an_image.append(node_names[np.where(node_names[:, -1] == pfai)[0]][0])
    
node_names_for_an_image = np.array(node_names_for_an_image)
#############--------------------------------

#print 'Number of crops :\n\n', len(crop_positions_for_an_image), '\n\n--------------\n'
#print 'Predictions     :\n\n', predictions_for_an_image, '\n\n--------------\n'
############----------------------new-----------------------
#print 'Node names      :\n\n', node_names_for_an_image, '\n\n--------------\n'
############--------------------------------------

In [ ]:
cropped_mean_subtracted_images_for_an_image = []
cropped_rescaled_images_for_an_image = []

for cpfai in crop_positions_for_an_image:
    _cropped = img_mean_subtracted[:, :, cpfai[0]:cpfai[1], cpfai[2]:cpfai[3]]
    _cropped = np.squeeze(_cropped, axis = 0).swapaxes(0, 2).swapaxes(0, 1)
    
    _cropped_rescaled = rescale_image(_cropped, scale = [224, 224])
    _cropped_rescaled = _cropped_rescaled.swapaxes(0, 2).swapaxes(1, 2)
    
    cropped_mean_subtracted_images_for_an_image.append(_cropped_rescaled)
    
    cropped_rescaled_images_for_an_image.append(rescale_image(img_rescaled[cpfai[0]:cpfai[1], cpfai[2]:cpfai[3]], 
                                                              scale = [224, 224]))
    
cropped_mean_subtracted_images_for_an_image = np.array(cropped_mean_subtracted_images_for_an_image)
cropped_rescaled_images_for_an_image = np.array(cropped_rescaled_images_for_an_image)

In [ ]:
#print 'Cropped Rescaled Images For an Image Shape        :', cropped_rescaled_images_for_an_image.shape
#print 'Cropped Mean Subtracted Images For an Image Shape :', cropped_mean_subtracted_images_for_an_image.shape

In [ ]:
net.blobs['data'].reshape(cropped_mean_subtracted_images_for_an_image.shape[0],
                          cropped_mean_subtracted_images_for_an_image.shape[1],
                          cropped_mean_subtracted_images_for_an_image.shape[2],
                          cropped_mean_subtracted_images_for_an_image.shape[3])

caffe_output = classify_images(net, cropped_mean_subtracted_images_for_an_image)

top_n_predictions_level1 = get_predictions(caffe_output, labels1, top_n, 'prob1')
top_n_predictions_level2 = get_predictions(caffe_output, labels2, top_n, 'prob2')
top_n_predictions_level3 = get_predictions(caffe_output, labels3, top_n, 'prob3')

In [ ]:
node_names_for_level1 = np.array(list(set(node_names_for_an_image[:, 0])))
node_names_for_level2 = np.array(list(set(node_names_for_an_image[:, 1])))
node_names_for_level3 = np.array(list(set(node_names_for_an_image[:, 2])))

#print node_names_for_level1
#print node_names_for_level2
#print node_names_for_level3
#print '---------------------------'
#print node_names_for_an_image

In [ ]:
crop_indexes_for_level1 = []

for i in xrange(len(node_names_for_level1)):
    
    #print '\n------- LABEL : {}'.format(node_names_for_level1[i])
    
    for k in xrange(5):
        
        out_index = list(np.where(top_n_predictions_level1[:, k, 0] == node_names_for_level1[i])[0])
        
        if out_index:
            #print '\nSelected : {}'.format(k)
            break
        
    if not out_index:
        out_index = None
        
    #print '\nCrop Indexes for {} :'.format(node_names_for_level1[i]), out_index
            
    crop_indexes_for_level1.append(out_index)
        
crop_indexes_for_level1 = np.array(crop_indexes_for_level1)

#print '\n--------------------------------------------------------------------------------------\n'
#print '\n------------- Selected Labels for Level 1\n\n', node_names_for_level1
#print '\n------------- Crop Indexes For Level 1:\n\n', crop_indexes_for_level1

In [ ]:
parent_nodes_for_level2 = []

for i in xrange(len(node_names_for_level2)):
    parent_nodes_for_level2.append(node_names_for_an_image[np.where(node_names_for_an_image[:, 1] == node_names_for_level2[i])[0]][:, 0][0])
    
parent_nodes_for_level2 = np.array(parent_nodes_for_level2)

#print '\nParent nodes for level2 :', parent_nodes_for_level2
#print '\nNode names for level2   :', node_names_for_level2

In [ ]:
crop_indexes_for_level2 = []

for i in xrange(len(node_names_for_level2)):
    
    #print '\n------- LABEL : {}'.format(node_names_for_level2[i])
    
    avaliable_crops = crop_indexes_for_level1[np.where(node_names_for_level1 == parent_nodes_for_level2[i])[0]][0]
    
    if avaliable_crops != None:
        
        avaliable_crops_numpy = np.array(avaliable_crops)
    
        #print '\nAvaliable crops for {} :'.format(node_names_for_level2[i]), avaliable_crops_numpy

        for k in xrange(5):

            out_index = list(np.where(top_n_predictions_level2[avaliable_crops_numpy, k, 0] == node_names_for_level2[i])[0])

            if out_index:
                out_index = list(avaliable_crops_numpy[out_index])
                #print '\nSelected : {}'.format(k)
                break

        if not out_index:
            out_index = None
            
    if avaliable_crops == None or out_index == None:
        
        #print '\nThere is no avaliable crop in Level 1'

        for k in xrange(5):

            out_index = list(np.where(top_n_predictions_level2[:, k, 0] == node_names_for_level2[i])[0])

            if out_index:
                #print '\nSelected : {}'.format(k)
                break

        if not out_index:
            out_index = None
        
    #print '\nCrop Indexes for {} :'.format(node_names_for_level2[i]), out_index
            
    crop_indexes_for_level2.append(out_index)
        
crop_indexes_for_level2 = np.array(crop_indexes_for_level2)

#print '\n--------------------------------------------------------------------------------------\n'
#print '\n------------- Selected Labels for Level 1\n\n', node_names_for_level2
#print '\n------------- Crop Indexes For Level 1:\n\n', crop_indexes_for_level2

In [ ]:
parent_nodes_for_level3 = []

for i in xrange(len(node_names_for_level3)):
    parent_nodes_for_level3.append(node_names_for_an_image[np.where(node_names_for_an_image[:, 2] == node_names_for_level3[i])[0]][:, 1][0])
    
parent_nodes_for_level3 = np.array(parent_nodes_for_level3)

#print '\nParent nodes for level3 :', parent_nodes_for_level3
#print '\nNode names for level3   :', node_names_for_level3

In [ ]:
crop_indexes_for_level3 = []

for i in xrange(len(node_names_for_level3)):
    
    #print '\n------- LABEL : {}'.format(node_names_for_level3[i])
    
    avaliable_crops = crop_indexes_for_level2[np.where(node_names_for_level2 == parent_nodes_for_level3[i])[0]][0]
    
    if avaliable_crops != None:
    
        avaliable_crops_numpy = np.array(avaliable_crops)

        #print '\nAvaliable crops for {} :'.format(node_names_for_level3[i]), avaliable_crops_numpy

        for k in xrange(5):

            out_index = list(np.where(top_n_predictions_level3[avaliable_crops_numpy, k, 0] == node_names_for_level3[i])[0])

            if out_index:
                out_index = list(avaliable_crops_numpy[out_index])
                #print '\nSelected : {}'.format(k)
                break

        if not out_index:
            out_index = None
            
    if avaliable_crops == None or out_index == None:
        
        #print '\nThere is no avaliable crop in Level 2'
        
        nnfl1 = parent_nodes_for_level2[np.where(node_names_for_level2 == parent_nodes_for_level3[i])[0]]
        
        avaliable_crops = crop_indexes_for_level1[np.where(node_names_for_level1 == nnfl1)[0]][0]
        
        if avaliable_crops != None:
            
            avaliable_crops_numpy = np.array(avaliable_crops)

            #print '\nAvaliable crops for {} :'.format(node_names_for_level3[i]), avaliable_crops_numpy

            for k in xrange(5):

                out_index = list(np.where(top_n_predictions_level3[avaliable_crops_numpy, k, 0] == node_names_for_level3[i])[0])

                if out_index:
                    out_index = list(avaliable_crops_numpy[out_index])
                    #print '\nSelected : {}'.format(k)
                    break

            if not out_index:
                out_index = None
                
        if avaliable_crops == None or out_index == None:
            
            #print '\nThere is no avaliable crop in Level 1'

            for k in xrange(5):

                out_index = list(np.where(top_n_predictions_level3[:, k, 0] == node_names_for_level3[i])[0])

                if out_index:
                    #print '\nSelected : {}'.format(k)
                    break

            if not out_index:
                out_index = None
            

    #print '\nCrop Indexes for {} :'.format(node_names_for_level3[i]), out_index
            
    crop_indexes_for_level3.append(out_index)
        
crop_indexes_for_level3 = np.array(crop_indexes_for_level3)

#print '\n--------------------------------------------------------------------------------------\n'
#print '\n------------- Selected Labels for Level 3\n\n', node_names_for_level3
#print '\n------------- Crop Indexes For Level 3:\n\n', crop_indexes_for_level3

In [ ]:
#print '----------------------------------------\n CLASSIFICATION RESULTS \n----------------------------------------'
#print_image(img_rescaled)
#print_predictions(predictions_for_an_image[-1], 'LEVEL 3')


#print '----------------------------------------\n LOCALIZATION RESULTS \n----------------------------------------'

predicted_bounding_boxes_for_an_image = []

for i in xrange(len(crop_indexes_for_level3)):
    
    #print '\n ----------------------------- LABEL      : {} ---------------------------'.format(node_names_for_level3[i])
    #print '\n ----------------------------- CROP INDEX : {} ---------------------------'.format(crop_indexes_for_level3[i])
    
    if crop_indexes_for_level3[i] is not None:
        bbox = np.round(np.mean(crop_positions_for_an_image[crop_indexes_for_level3[i]], axis = 0)).astype('int')
        #for ci in crop_indexes_for_level3[i]:
        #    print 'Crop Index : {}'.format(ci)
        #    print_image(cropped_rescaled_images_for_an_image[ci])
        #print '-------------------------------- MEAN -----------------------------------'
        #print_image(img_rescaled[bbox[0] : bbox[1], bbox[2] : bbox[3]])
        
    else:
        bbox = None
        
    predicted_bounding_boxes_for_an_image.append(bbox)
    
predicted_bounding_boxes_for_an_image = np.array(predicted_bounding_boxes_for_an_image)
        
        
#print '----------------------------------------\n CLASSIFICATION RESULTS \n----------------------------------------'
#print_image(img_rescaled)
#print_predictions(predictions_for_an_image[-1], 'LEVEL 3')


#print '----------------------------------------\n LOCALIZATION RESULTS \n----------------------------------------'
#for i in xrange(len(crop_indexes_for_level3)):
    
#    print '\n ----------------------------- LABEL      : {} ---------------------------'.format(node_names_for_level3[i])
#    print '\n ----------------------------- CROP INDEX : {} ---------------------------'.format(crop_indexes_for_level3[i])
    
#    if crop_indexes_for_level3[i] is not None:
#        max_index = crop_indexes_for_level3[i][np.argmax(top_n_predictions_level3[crop_indexes_for_level3[i]][np.where(top_n_predictions_level3[crop_indexes_for_level3[i]] == 
#                                                                        node_names_for_level3[i])[:2]][:, 1].astype('float32'))]
    
#    print_image(cropped_rescaled_images_for_an_image[max_index])
#    print '\nCrop Position : ', crop_positions_for_an_image[max_index]

In [ ]:
print predicted_bounding_boxes_for_an_image